# Problem Statement
In a competitve market like restaurant industry, understanding the factors that influences business success is crucial for stakeholders. Utilizing yelp dataset, this project aims relationship between user engagement (reviews, tips, check-ins) and business success metrics (review count, ratings) for restaurants.

### Research Objectives
1. **Quantify the correlation between user engagement (reviews, tips, check-ins) and review count/average star rating**: This will help us determine that with higher user engagement experience a corresponding increase in reviews and ratings.

2. **Analyze the impact of sentiment on review count and average star rating**: We will investigate if positive sentiment in reviews result into higher star ratings and potentially influences total number of reviews left.

3. **Time Trends in User Engagement**: We will explore if consistent user engagement over time is a stonger indicator of lont-term success compared to sporadic bursts of activity.

### Hypothesis Testing
- Higher levels of user engagement (more reviews, tips and check-ins) correlate with higher review count and ratings for restaurant
- Positive sentiment expressed in review and tips contributes to higher overall ratings and review count for restaurant
- Consistent engagement over time is positively associated with sustained business success for restaurants. 

## Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np
import sqlite3
import folium
from geopy.geocoders import Nominatim
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
from sqlalchemy import create_engine

In [3]:
# Function to read large JSON files efficiently
def read_large_json(file_path, chunk_size=100000):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            data.append(json.loads(line))
            if (i + 1) % chunk_size == 0:
                print(f"Processed {i+1} lines from {file_path}...")

    return pd.DataFrame(data)

# Base directory for all datasets
base_path = r'C:\Users\Admin\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\\'

# Load each dataset efficiently
print("Loading Business Data...")
business_df = read_large_json(base_path + 'yelp_academic_dataset_business.json')

print("Loading Check-in Data...")
checkin_df = read_large_json(base_path + 'yelp_academic_dataset_checkin.json')

print("Loading Review Data...")
review_df = read_large_json(base_path + 'yelp_academic_dataset_review.json')

print("Loading Tip Data...")
tip_df = read_large_json(base_path + 'yelp_academic_dataset_tip.json')

print("Loading User Data...")
user_df = read_large_json(base_path + 'yelp_academic_dataset_user.json')

print("✅ All datasets loaded successfully!")


Loading Business Data...
Processed 100000 lines from C:\Users\Admin\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\\yelp_academic_dataset_business.json...
Loading Check-in Data...
Processed 100000 lines from C:\Users\Admin\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\\yelp_academic_dataset_checkin.json...
Loading Review Data...
Processed 100000 lines from C:\Users\Admin\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\\yelp_academic_dataset_review.json...
Processed 200000 lines from C:\Users\Admin\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\\yelp_academic_dataset_review.json...
Processed 300000 lines from C:\Users\Admin\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\\yelp_academic_dataset_review.json...
Processed 400000 lines from C:\Users\Admin\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\\yelp_academic_dataset_review.json...
Processed 500000 lines from C:\Users\Admin\.cache\kagglehub\

In [4]:
print(business_df.shape)
print(checkin_df.shape)
print(review_df.shape)
print(tip_df.shape)
print(user_df.shape)

(150346, 14)
(131930, 2)
(6990280, 9)
(908915, 5)
(1987897, 22)


In [5]:
business_df.drop(['attributes','hours'], axis=1, inplace=True)

In [6]:
# Creating database connection
engine = create_engine('sqlite:///yelp.db')

def load_dataframe(df, table_name, engine):
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Load each DataFrame into separate table
load_dataframe(business_df, 'business', engine)
load_dataframe(review_df, 'review', engine)
load_dataframe(user_df, 'user', engine)
load_dataframe(tip_df, 'tip', engine)
load_dataframe(checkin_df, 'checkin', engine)

In [8]:
# Creating database connection
conn = sqlite3.connect('yelp.db')

In [9]:
tables = pd.read_sql_query("SELECT name from sqlite_master where type = 'table'",conn)

In [10]:
tables

,name
0,business
1,review
2,user
3,tip
4,checkin


In [12]:
for table in tables['name']:
    print(table)
    display(pd.read_sql_query(f"SELECT * from {table} limit 1", conn))

business


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,"Doctors, Traditional Chinese Medicine, Naturop..."


review


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11


user


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180


tip


,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0


checkin


,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."


# Data Analysis

In [13]:
pd.read_sql_query("select count(*) from business",conn)

,count(*)
0,150346


In [39]:
display(pd.read_sql_query("""
                  select count(*) from business where is_open=1 and lower(categories) like '%restaurant%'
                  """, conn))
business_id = pd.read_sql_query("""
                  select business_id from business where is_open=1 and lower(categories) like '%restaurant%'
                  """, conn)

,count(*)
0,35004


In [48]:
# What is the descriptive stats for review count and star rating for business ?

pd.read_sql_query(f"""
                  select 
                  avg(review_count) as avg_review_count, 
                  min(review_count) as min_review_count, 
                  max(review_count) as max_review_count,
                  (select review_count from business order by review_count limit 1 offset 
                    (select count(*) from business)/2) as median_review_count,
                  
                  avg(stars) as avg_stars_count, 
                  min(stars) as min_stars_count, 
                  max(stars) as max_stars_count,
                  (select stars from business order by stars limit 1 offset 
                    (select count(*) from business)/2) as median_stars_count
                  
                  from business 
                  where business_id IN {tuple(business_id['business_id'])}
                  """, conn).transpose()

,0
avg_review_count,104.097789
min_review_count,5.000000
max_review_count,7568.000000
median_review_count,15.000000
avg_stars_count,3.523969
min_stars_count,1.000000
max_stars_count,5.000000
median_stars_count,3.500000


Since median review count and max are very different, there are outliers, but outlier can have meaning when we are trying to find out highest rated restaurants. So we will create a function that removes outliers when we need to.

***We will use IQR fro outliers remove***

In [54]:
def remove_outlier(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    IQR = q3 - q1
    lower_bound = q1 - 1.5*IQR
    upper_bound = q3 + 1.5*IQR

    df = df[(df[col] >= lower_bound * (df[col] <= upper_bound))]
    
    return df

In [55]:
trial = pd.read_sql_query("""
                  select business_id, review_count from business where is_open=1 and lower(categories) like '%restaurant%'
                  """, conn)

In [56]:
trial = remove_outlier(trial, 'review_count')

In [57]:
pd.read_sql_query(f"""
                  select 
                  avg(review_count) as avg_review_count, 
                  min(review_count) as min_review_count, 
                  max(review_count) as max_review_count,
                  (select review_count from business order by review_count limit 1 offset 
                    (select count(*) from business)/2) as median_review_count,
                  
                  avg(stars) as avg_stars_count, 
                  min(stars) as min_stars_count, 
                  max(stars) as max_stars_count,
                  (select stars from business order by stars limit 1 offset 
                    (select count(*) from business)/2) as median_stars_count
                  
                  from business 
                  where business_id IN {tuple(trial['business_id'])}
                  """, conn).transpose()

,0
avg_review_count,104.097789
min_review_count,5.000000
max_review_count,7568.000000
median_review_count,15.000000
avg_stars_count,3.523969
min_stars_count,1.000000
max_stars_count,5.000000
median_stars_count,3.500000
